In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from surrogate import ImageSurrogate

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
import tensorflow_datasets as tfds

from datetime import datetime
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from tensorflow.keras.layers import (Input, Layer, Dense, Conv2D)
from tensorflow.keras.models import Model

In [5]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [6]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Save Dirs

In [7]:
save_dir = 'cxplain'
model_dir = os.path.join(os.getcwd(), save_dir, 'results')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

## Load Data

In [8]:
import itertools
x_train=[]
y_train=[]
for i in range(10):
    x_train.append(np.load(os.path.join(model_dir,'x_train_'+str(i)+'.npy'), allow_pickle=True))
    y_train.append(np.load(os.path.join(model_dir,'y_train_'+str(i)+'.npy'), allow_pickle=True))
x_train=np.array(list(itertools.chain(*x_train)))
y_train=np.array(list(itertools.chain(*y_train)))
x_val=np.load(os.path.join(model_dir,'x_val.npy'), allow_pickle=True)
y_val=np.load(os.path.join(model_dir,'y_val.npy'), allow_pickle=True)
x_test=np.load(os.path.join(model_dir,'x_test.npy'), allow_pickle=True)
y_test=np.load(os.path.join(model_dir,'y_test.npy'), allow_pickle=True)

## Load Model

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50

INPUT_SHAPE = (224,224,3)

base_model = ResNet50(
    include_top=True, weights='imagenet', 
    input_shape=INPUT_SHAPE
)
base_model.trainable = False

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210511_21_28_36/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# CXPlain

In [16]:
import sys
sys.path.insert(0, '../../../cxplain')
from tensorflow.python.keras.losses import categorical_crossentropy
from cxplain.backend.masking.zero_masking import ZeroMasking
from cxplain.backend.tensorflow_cxplain import TensorflowCXPlain as CXPlain

In [17]:
downsample_factors = (16, 16)
batch_size = 32
learning_rate=0.001
masking_operation = ZeroMasking()
loss = categorical_crossentropy

In [18]:
## ResNet Explainer Model Builder Class
from cxplain.backend.model_builders.base_model_builder import BaseModelBuilder
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model


class ResNetModelBuilder(BaseModelBuilder):
    def __init__(self, downsample_factors, 
                 callbacks=list([]), early_stopping_patience=12,
                 batch_size=64, num_epochs=100, validation_fraction=0.1, shuffle=True,
                 learning_rate=0.0001, optimizer=None, verbose=0):
        super(ResNetModelBuilder, self).__init__(callbacks, early_stopping_patience, batch_size, num_epochs,
                                                 validation_fraction, shuffle, learning_rate, optimizer, verbose)
        self.downsample_factors = downsample_factors
        self.num_output_channels = 1

    def build(self, input_layer):
        downsampling_factor = int(np.prod(self.downsample_factors))
        last_layer = input_layer
        
        base_model = ResNet50(
            include_top=False, weights='imagenet', 
            input_shape=(224,224,3)
        )
        base_model = Model(base_model.input, base_model.get_layer('conv4_block3_2_conv').output)
        base_model.trainable = True

        last_layer = base_model(last_layer)

        # Learn Phi 
        last_layer = Conv2D(1, 1)(last_layer)
        
        return last_layer

In [19]:
model_input = Input((224,224,3))
base_model = ResNet50(
            include_top=False, weights='imagenet', 
            input_shape=(224,224,3)
        )
base_model = Model(base_model.input, base_model.get_layer('conv4_block3_2_conv').output)
base_model.trainable = True

last_layer = base_model(model_input)

# Learn Phi 
last_layer = Conv2D(1, 1)(last_layer)

m = Model(model_input, last_layer)

In [20]:
model_builder = ResNetModelBuilder(downsample_factors, verbose=1, batch_size=32,
                                   learning_rate=0.001)

explainer = CXPlain(model, model_builder, masking_operation, loss, 
                    num_models=1, downsample_factors=downsample_factors, flatten_for_explained_model=False)

In [21]:
t = time.time()
explainer.fit(x_train, y_train)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)

  0%|          | 0/296 [00:00<?, ?it/s]

Created Masked Data
Fitting Explainer Model
Train on 8522 samples, validate on 947 samples
Epoch 1/100
8522/8522 [==============================] - 73s 9ms/sample - loss: 1.0599 - lambda_6_loss: 1.0554 - all_loss: 0.0019 - lambda_7_loss: 0.0020 - val_loss: 1.0563 - val_lambda_6_loss: 1.0520 - val_all_loss: 0.0015 - val_lambda_7_loss: 0.0017
Epoch 2/100
8522/8522 [==============================] - 31s 4ms/sample - loss: 1.0295 - lambda_6_loss: 1.0251 - all_loss: 0.0019 - lambda_7_loss: 0.0020 - val_loss: 1.0498 - val_lambda_6_loss: 1.0413 - val_all_loss: 0.0015 - val_lambda_7_loss: 0.0016
Epoch 3/100
8522/8522 [==============================] - 31s 4ms/sample - loss: 1.0231 - lambda_6_loss: 1.0201 - all_loss: 0.0019 - lambda_7_loss: 0.0020 - val_loss: 1.0311 - val_lambda_6_loss: 1.0279 - val_all_loss: 0.0015 - val_lambda_7_loss: 0.0016
Epoch 4/100
8522/8522 [==============================] - 31s 4ms/sample - loss: 1.0111 - lambda_6_loss: 1.0073 - all_loss: 0.0019 - lambda_7_loss: 0.0020

# Explain w/ CXPlain

### Load Images

In [22]:
images_dir = os.path.join(os.getcwd(), 'images')
images = np.load(os.path.join(images_dir, 'processed_images.npy'), allow_pickle=True)

### Explain

In [23]:
t = time.time()
explanations = explainer.explain(images)
explaining_time = time.time() - t

### Save

In [24]:
with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)
    
explanations.dump(os.path.join(model_dir, 'explanations.npy'))